In [ ]:
import requests
from datetime import datetime, timedelta
import pandas as pd
from dotenv import load_dotenv
import os
from tqdm import tqdm
import zipfile
from pathlib import Path
import glob
import jquantsapi
import inspect
from bs4 import BeautifulSoup
from io import StringIO
import re

In [ ]:
document_list_df = pd.read_excel('document_list.xlsx')

In [ ]:
debug = False
# debug = True

def find_all_regex_in_dataframe(df, pattern):
    """
    DataFrameから正規表現パターンにマッチする全てのセルを検索
    
    Parameters:
    -----------
    df : pandas.DataFrame
        検索対象のDataFrame
    pattern : str
        正規表現パターン
    
    Returns:
    --------
    list
        マッチした全てのセルの位置のリスト [(row, col), ...]
    """
    positions = []
    for row in range(len(df)):
        for col in range(len(df.columns)):
            cell_value = str(df.iloc[row, col])

            # debug
            # if pattern ==  r'(?:帳簿価額|土地)\s?[\(（]百万円[\)）]':
            #     print(row, col, pattern, cell_value)
            # print(cell_value, type(cell_value))
            
            if re.search(pattern, cell_value):
                positions.append((row, col))
    return positions

all_lands = []

for i, document_info in tqdm(document_list_df.iterrows(), total=len(document_list_df), desc='読み込み中', unit='ファイル'):
    doc_id = document_info['docID']
    sec_code = document_info['secCode']
    filer_name = document_info['filerName']

    # debug
    if debug:
        if sec_code != '32860':
            continue

    
    file_pattern = f"documents/{document_info['docID']}/XBRL/PublicDoc/0103010_honbun_jpcrp030000*.htm"
    files = glob.glob(file_pattern)
    if len(files) == 0:
        continue
    file = files[0]
    
    with open(file, 'r', encoding='utf-8') as file:
        html_content = file.read()
    
    soup = BeautifulSoup(html_content, 'html.parser')
    ix_tag = soup.find('ix:nonnumeric', {'name': 'jpcrp_cor:MajorFacilitiesTextBlock'})

    try:
        df_list = pd.read_html(StringIO(str(ix_tag)), header=0)
    except ValueError:
        continue
    
    lands = []
    
    for df in df_list:

        # ヘッダー行を最初の行に戻す
        header_row = pd.DataFrame([df.columns.tolist()])
        header_row.columns = range(len(df.columns))
        df.columns = range(len(df.columns))
        df = pd.concat([header_row, df], ignore_index=True)
        
        # debug
        if debug:
            display(df)

        if not find_all_regex_in_dataframe(df, r'土地'):
            continue

        price_unit = None
        area_unit = 1
        loc_type = None
        price_type = None
        area_type = None
        start_row = 0

        # 所在地
        if not loc_type:
            results = find_all_regex_in_dataframe(df, r'^(?:主な)?(?:投資不動産内容|事業所名|子会社名|名称)(?:及び(?:主な)?(?:事業所名|子会社名|名称))?\s?[\(（](?:主な)?所在地[\)）]$')
            if results:
                start_row = max(start_row, results[-1][0] + 1)
                loc_type = 'x(a)'
                loc_col = results[-1][1]
        if not loc_type:
            results = find_all_regex_in_dataframe(df, r'^(?:主な)?所在地$')
            if results:
                start_row = max(start_row, results[-1][0] + 1)
                loc_type = 'a'
                loc_col = results[-1][1]

        # 価格と面積
        if not price_type or not area_type:
            results = find_all_regex_in_dataframe(df, r'^土地\s?([\(（](?:百万円|千円)[\)）])?\s?[\(（]面積(?:㎡|千㎡)[\)）]$')
            if results:
                start_row = max(start_row, results[-1][0] + 1)
                price_type = 'a(x)'
                price_col = results[-1][1]
                area_type = 'x(a)'
                area_col = results[-1][1]
                if '千㎡' in df.iloc[results[-1][0], results[-1][1]]:
                    area_unit = 1000

        # 面積のみ
        # 「土地」セルの下に「金額」セルがある
        if not price_type:
            # まず「土地」セルを探す
            land_results = find_all_regex_in_dataframe(df, r'^土地$')
            for land_row, land_col in land_results:
                # その下の行で同じ列に「金額」があるか確認
                if land_row + 1 < len(df):
                    next_cell = str(df.iloc[land_row + 1, land_col])
                    if re.search(r'金額', next_cell):
                        start_row = max(start_row, land_row + 2)  # データは「価格」の次の行から
                        price_type = 'a'
                        price_col = land_col
                        break
        # 面積セル
        if not price_type or not area_type:
            results = find_all_regex_in_dataframe(df, r'^面積[\(（]?(?:㎡|千㎡)[\)）]?$')
            if results:
                start_row = max(start_row, results[-1][0] + 1)
                area_type = 'a'
                area_col = results[-1][1]
                if '千㎡' in df.iloc[results[-1][0], results[-1][1]]:
                    area_unit = 1000

        # 価格のみ - 「土地」セルの下に「面積」セルがある
        if not area_type:
            # まず「土地」セルを探す
            land_results = find_all_regex_in_dataframe(df, r'^土地$')
            for land_row, land_col in land_results:
                # その下の行で同じ列に「面積」があるか確認
                if land_row + 1 < len(df):
                    next_cell = str(df.iloc[land_row + 1, land_col])
                    if re.search(r'^面積[\(（]?(?:㎡|千㎡)[\)）]?$', next_cell):
                        start_row = max(start_row, land_row + 2)  # データは「面積」の次の行から
                        area_type = 'a'
                        area_col = land_col
                        if '千㎡' in next_cell:
                            area_unit = 1000
                        break
        # 価格のみ - 「帳簿価格」セルの下に「土地」セルがある
        if not price_type:
            # まず「帳簿価格」セルを探す
            book_value_results = find_all_regex_in_dataframe(df, r'^帳簿価額([\(（](?:百万円|千円)[\)）])?.*$')
            for book_value_row, book_value_col in book_value_results:
                # その下の行で同じ列に「土地」があるか確認
                if book_value_row + 1 < len(df):
                    next_cell = str(df.iloc[book_value_row + 1, book_value_col])
                    if re.search(r'土地', next_cell):
                        start_row = max(start_row, book_value_row + 2)  # データは「土地」の次の行から
                        price_type = 'a'
                        price_col = book_value_col
                        # 単位の確認
                        if not price_unit:
                            unit_cell = df.iloc[book_value_row, book_value_col]
                            if re.search(r'[\(（]百万円[\)）]', str(unit_cell)):
                                price_unit = 1000000
                            elif re.search(r'[\(（]千円[\)）]', str(unit_cell)):
                                price_unit = 1000
                        break
                        
        # 単位
        if not price_unit:
            results = find_all_regex_in_dataframe(df, r'^(?:帳簿価額|土地)\s?[\(（]百万円[\)）]$')
            if results:
                price_unit = 1000000
        if not price_unit:
            results = find_all_regex_in_dataframe(df, r'^(?:帳簿価額|土地)\s?[\(（]千円[\)）]$')
            if results:
                price_unit = 1000

        # debug
        if debug:
            print(loc_type, price_type, area_type)

        # 値の読み取り
        for row in range(start_row, len(df)):
            loc = None
            if loc_type == 'x(a)':
                match = re.search(r'^.+\s?[\(（](?!注\d*)([^\)）]+)[\)）]$', str(df.at[row, loc_col]))
                if match:
                    loc = match.group(1)
            if loc_type == 'a':
                loc = str(df.at[row, loc_col])
        
            price = None
            if price_type == 'a(x)':
                # 最初のセルの内容を取得
                first_cell = str(df.at[row, price_col])
                
                # カッコが含まれていない場合、次のセルの内容と結合
                if first_cell.replace(',', '').replace('.', '').isdigit():
                    # 次のセルの内容を取得（行末でないことを確認）
                    if row + 1 < len(df):
                        second_cell = str(df.at[row + 1, price_col])
                        first_cell += second_cell
                
                # 結合後のセルから値を抽出
                match = re.search(r'^(.+)\s?[\(（](?!注\d*)[^\)）]+[\)）]$', first_cell)
                if match:
                    price = match.group(1)
            elif price_type == 'a':
                price = str(df.at[row, price_col])
        
            area = None
            if area_type == 'x(a)':
                # 最初のセルの内容を取得
                first_cell = str(df.at[row, area_col])
                
                # カッコが含まれていない場合、次のセルの内容と結合
                if first_cell.replace(',', '').replace('.', '').isdigit():
                    # 次のセルの内容を取得（行末でないことを確認）
                    if row + 1 < len(df):
                        second_cell = str(df.at[row + 1, area_col])
                        first_cell += second_cell
                
                # 結合後のセルから値を抽出
                match = re.search(r'^.+\s?[\(（](?!注\d*)([^\)）]+)[\)）]$', first_cell)
                if match:
                    area = match.group(1)
            elif area_type == 'a':
                match = re.search(r'^(?:[\(（].+[\)）])?(.*)\s?(?:[\(（].+[\)）])?$', str(df.at[row, area_col]))
                if match:
                    area = match.group(1)
        
            lands.append({
                'doc_id': doc_id,
                'sec_code': sec_code,
                'filer_name': filer_name,
                'loc': loc,
                'price': price,
                'area': area,
                'price_unit': price_unit,
                'area_unit': area_unit,
            })
    
    all_lands.extend(lands)

In [ ]:
lands_df = pd.DataFrame(all_lands)
lands_df

In [ ]:
lands_df.to_excel('lands_uncleansed.xlsx', index=None)

In [ ]:
lands_df['price'] = pd.to_numeric(lands_df['price'].str.replace(',', ''), errors='coerce')
lands_df['area'] = pd.to_numeric(lands_df['area'].str.replace(',', ''), errors='coerce')
lands_df = lands_df.dropna()

In [ ]:
lands_df.to_excel('lands_cleansed.xlsx', index=None)